In [53]:
import numpy as np

class Queen:
    def __init__(self, N):
        self.N = N
        self.queen_loc = dict()
        self.initialize = False
        self.chess_board = [[0]*self.N for _ in range(self.N)]

    def add_queen(self):
        if self.initialize == False:
            number_Q = 0
            while True:
                flag = 0
                r = np.random.randint(self.N)
                c = np.random.randint(self.N)
                for q in self.queen_loc:
                    row, col = self.queen_loc[q]
                    if (r==row and c==col) or (c==col):
                        flag = 1
                if flag == 0:
                    Q = f"Q{number_Q}"
                    if Q not in self.queen_loc:
                        self.queen_loc[Q] = []
                    self.queen_loc[Q].append(r)
                    self.queen_loc[Q].append(c)
                    self.chess_board[r][c] = Q
                    number_Q += 1
                if number_Q == self.N:
                    break
            self.initialize = True

    def get_neighbor(self, row, col):
        neighbor = []
        if 0<=row-1<self.N and self.chess_board[row-1][col]==0:
            neighbor.append([row-1,col])
        if 0<=row+1<self.N and self.chess_board[row+1][col]==0:
            neighbor.append([row+1,col])
        return neighbor

    def print_Queen(self):
        print(self.chess_board)
        for Q in self.queen_loc:
            print(f'{Q}->{self.queen_loc[Q]}')

In [54]:
def conflict(r1,c1,r2,c2):
    if r1 == r2:
        return True
    if c1 == c2:
        return True
    if r1+c1 == r2+c2:
        return True
    if r1-c1 == r2-c2:
        return True
    return False

In [55]:
def get_conflict(Q,state):
    count = 0
    for q in state:
        if q is not  Q:
            r1, c1 = state[Q]
            r2, c2 = state[q]
            if conflict(r1,c1,r2,c2):
                count += 1
    return count

In [56]:
def calc_cost(state):
    cost = 0
    max = -999
    maxQ = None
    for Q in state:
        q_cost = get_conflict(Q,state)
        cost += q_cost
        if q_cost>max:
            max = q_cost
            maxQ = Q
    return cost//2,max,maxQ

In [57]:
def goal_test(state):
    if calc_cost(state)[0] == 0:
        return True
    else:
        return False

In [58]:
from re import X

def simulated_annealing(board, temperature, reduction_rate):

    current_state = board.queen_loc.copy()
    stop_taking_bad_moves = 0.001

    while temperature > stop_taking_bad_moves:
        neighbors = []
        for x in current_state:
            r, c = current_state[x]
            for new_pos in board.get_neighbor(r, c):
                new_state = current_state.copy()
                new_state[x] = new_pos
                neighbors.append((new_state, calc_cost(new_state)[0]))
        if not neighbors:
            break
        neighbors.sort(key=lambda x: x[1])
        next_state = neighbors[0][0]
        delta_E = calc_cost(next_state)[0] - calc_cost(current_state)[0]
        if delta_E < 0 or np.random.uniform(0, 1) < np.exp(-delta_E / temperature):
            current_state = next_state
        temperature *= reduction_rate
    return current_state

In [59]:
queen = Queen(4)
queen.add_queen()
queen.print_Queen()

temperature = 1000
reduction_rate = 0.99

result = simulated_annealing(queen, temperature, reduction_rate)

print("\nSolution:")
for x in result:
    print(f'{x}->{result[x]}')

[[0, 0, 0, 0], [0, 'Q3', 0, 0], ['Q1', 0, 'Q2', 'Q0'], [0, 0, 0, 0]]
Q0->[2, 3]
Q1->[2, 0]
Q2->[2, 2]
Q3->[1, 1]

Solution:
Q0->[1, 3]
Q1->[3, 0]
Q2->[3, 2]
Q3->[0, 1]
